In [184]:
import pandas as pd
import os
import json
import re

In [185]:
dataset_path='./dev_rand_split.jsonl'
exp_output = './exp2.txt'

In [186]:
false_QS = []
stem = 'A revolving door is convenient for two direction travel, but it also serves as a security measure at a what?'

In [187]:
with open(dataset_path) as f:
    lines = f.readlines()
    for line in lines:
        json_res = json.JSONDecoder().raw_decode(line)[0]
        if (json_res["question"]["stem"]==stem):
            false_QS.append(json_res)
f.close()            

In [188]:
false_QS

[{'answerKey': 'A',
  'id': '1afa02df02c908a558b4036e80242fac',
  'question': {'question_concept': 'revolving door',
   'choices': [{'label': 'A', 'text': 'bank'},
    {'label': 'B', 'text': 'library'},
    {'label': 'C', 'text': 'department store'},
    {'label': 'D', 'text': 'mall'},
    {'label': 'E', 'text': 'new york'}],
   'stem': 'A revolving door is convenient for two direction travel, but it also serves as a security measure at a what?'}}]

In [189]:
question = []
pred_before= []
pred_after=[]
pred=[]
GT = []

In [190]:
with open(exp_output) as e:
    lines = e.readlines()
    for line in lines:
        found = re.search('^Q: (.*) Answer Choices:',line)
        if found:
            question.append(found.group(1))
e.close()
print(question[:3])

['What are candles good for eliminating?', 'If there is a pond with trees around it, where it it likely located?', 'Reading newspaper one of many ways to practice your what?']


In [191]:
len(question)

1221

In [192]:
#pred_before
with open(exp_output) as e:
    lines = e.readlines()
    for line in lines:
        found = re.search('^pred_before :  (.*)',line)
        if found:
            pred_before.append(found.group(1))
e.close()
print(pred_before[:3])

['probably (A), (B), (C), or (D).', 'C.', 'A.']


In [193]:
len(pred_before)

1221

In [194]:
#pred_after
with open(exp_output) as e:
    lines = e.readlines()
    for line in lines:
        found = re.search('^pred_after : (.*)',line)
        if found:
            pred_after.append(found.group(1))
e.close()
print(pred_after[:3])

['A', 'C', 'A']


In [195]:
len(pred_after)

1221

In [196]:
#pred
with open(exp_output) as e:
    lines = e.readlines()
    for line in lines:
        found = re.search('^pred : (.*)',line)
        if found:
            pred.append(found.group(1))
e.close()
print(pred[:3])

['A', 'C', 'A']


In [197]:
len(pred)

1221

In [198]:
#GT
with open(exp_output) as e:
    lines = e.readlines()
    for line in lines:
        found = re.search('^GT : (.*)',line)
        if found:
            GT.append(found.group(1))
e.close()
print(GT[:3])

['E', 'C', 'A']


In [199]:
len(GT)

1221

In [200]:
#A
A=[]
with open(exp_output) as e:
    lines = e.readlines()
    for line in lines:
        found = re.search('^A: (.*)',line)
        if found:
            A.append(found.group(1))
e.close()           
print(A[:3])

["Let's think step by step. Candles are often used for light, so (E) dark is probably not the answer. Candles are also used for decoration or to make a place smell nice, so (A) shelf, (B) board, (C) church, and (D) table are all possible answers. ", "Let's think step by step. A pond is likely to be located near trees because trees need water to survive. Therefore, the most likely location for a pond with trees around it is in a forest. ", 'Let\'s think step by step. The first word is "reading." This narrows down our choices to A and B. The second word is "newspaper." This points us more towards A, because option B doesn\'t make much sense. The last word is "practice." This means that the answer is probably A, because option B is more of a skill than something you practice. ']


In [201]:
len(A)

1223

In [202]:
therefore = []
with open(exp_output) as e:
    lines = e.readlines()
    for line in lines:
        found = re.search('^Therefore, among A through E, (.*)',line)
        if found:
            therefore.append(found.group(1))
e.close()            
print(therefore[:3])

['the answer is probably (A), (B), (C), or (D).', 'the answer is C.', 'the answer is A.']


In [203]:
len(therefore)

1221

In [204]:
cols = ["question","A","therefore","pred_before","pred_after","pred","GT"]
response_data = pd.DataFrame(columns=cols)
response_data["question"]=question
#response_data["A"]=A
response_data["therefore"]=therefore
response_data["pred_before"]=pred_before
response_data["pred_after"]=pred_after
response_data["pred"]=pred
response_data["GT"]=GT
response_data.head()

,question,A,therefore,pred_before,pred_after,pred,GT
0,What are candles good for eliminating?,NaN,"the answer is probably (A), (B), (C), or (D).","probably (A), (B), (C), or (D).",A,A,E
1,"If there is a pond with trees around it, where...",NaN,the answer is C.,C.,C,C,C
2,Reading newspaper one of many ways to practice...,NaN,the answer is A.,A.,A,A,A
3,What happens when to ice when it is in the sun?,NaN,the answer is (B).,(B).,B,B,B
4,James loved to surf but he wasn't good at it. ...,NaN,the answer is A.,A.,A,A,A


In [205]:
len(question)

1221

In [206]:
len(pred_before)

1221

In [207]:
len(pred_after)

1221

In [208]:
len(pred)

1221

In [209]:
len(response_data.question)

1221

In [210]:
len(response_data)

1221

In [211]:
response_data['correct'] = response_data.apply(lambda row: True if row.pred==row.GT else False,axis=1)
response_data.head()

,question,A,therefore,pred_before,pred_after,pred,GT,correct
0,What are candles good for eliminating?,NaN,"the answer is probably (A), (B), (C), or (D).","probably (A), (B), (C), or (D).",A,A,E,False
1,"If there is a pond with trees around it, where...",NaN,the answer is C.,C.,C,C,C,True
2,Reading newspaper one of many ways to practice...,NaN,the answer is A.,A.,A,A,A,True
3,What happens when to ice when it is in the sun?,NaN,the answer is (B).,(B).,B,B,B,True
4,James loved to surf but he wasn't good at it. ...,NaN,the answer is A.,A.,A,A,A,True


with open(dataset_path) as f:
    lines = f.readlines()
    for line in lines:
        json_res = json.JSONDecoder().raw_decode(line)[0]
        if (json_res["question"]["stem"] not in list(response_data.question)):
            wrong_qs.append(json_res)

In [217]:
response_data[["question", "therefore","pred_before","pred_after","pred","GT","correct"]].head()

,question,therefore,pred_before,pred_after,pred,GT,correct
0,What are candles good for eliminating?,"the answer is probably (A), (B), (C), or (D).","probably (A), (B), (C), or (D).",A,A,E,False
1,"If there is a pond with trees around it, where...",the answer is C.,C.,C,C,C,True
2,Reading newspaper one of many ways to practice...,the answer is A.,A.,A,A,A,True
3,What happens when to ice when it is in the sun?,the answer is (B).,(B).,B,B,B,True
4,James loved to surf but he wasn't good at it. ...,the answer is A.,A.,A,A,A,True


In [218]:
mask_incorrect_qs = response_data.correct==False
len(mask_incorrect_qs[mask_incorrect_qs==True])

442

In [214]:
wrong_qs=[]
with open(dataset_path) as f:
    lines = f.readlines()
    for q in response_data[mask_incorrect_qs].question:

        for line in lines:
            json_res = json.JSONDecoder().raw_decode(line)[0]
            if (json_res["question"]["stem"]==q):
                wrong_qs.append(json_res)
print(wrong_qs[:3])
f.close()
with open("wrong_qs.jsonl", "w") as file:
    for qs in wrong_qs:
        json.dump(qs, file)
        file.write('\n')

#Using json.dump() to write the data into a JSON file.    

#    json.dump(wrong_qs, file,separators=)
file.close()

[{'answerKey': 'E', 'id': '83808e92381b2e5f4cdf55d1391645ae', 'question': {'question_concept': 'candle', 'choices': [{'label': 'A', 'text': 'shelf'}, {'label': 'B', 'text': 'board'}, {'label': 'C', 'text': 'church'}, {'label': 'D', 'text': 'table'}, {'label': 'E', 'text': 'dark'}], 'stem': 'What are candles good for eliminating?'}}, {'answerKey': 'E', 'id': '13fc28f53423a9b3a656c9431df1b3b5', 'question': {'question_concept': 'kissing', 'choices': [{'label': 'A', 'text': 'sexual stimulation'}, {'label': 'B', 'text': 'herpes'}, {'label': 'C', 'text': 'headache'}, {'label': 'D', 'text': 'catch cold'}, {'label': 'E', 'text': 'happiness'}], 'stem': 'What is the thing that is agitated in your head when kissing?'}}, {'answerKey': 'A', 'id': '7905b9f4ba503b0ce13b576808e99c42', 'question': {'question_concept': 'toy car', 'choices': [{'label': 'A', 'text': "child's room"}, {'label': 'B', 'text': "boy's bedroom"}, {'label': 'C', 'text': 'own home'}, {'label': 'D', 'text': 'toy store'}, {'label': 

In [215]:
len(wrong_qs)

442

In [216]:
wrong_qs[:10]

[{'answerKey': 'E',
  'id': '83808e92381b2e5f4cdf55d1391645ae',
  'question': {'question_concept': 'candle',
   'choices': [{'label': 'A', 'text': 'shelf'},
    {'label': 'B', 'text': 'board'},
    {'label': 'C', 'text': 'church'},
    {'label': 'D', 'text': 'table'},
    {'label': 'E', 'text': 'dark'}],
   'stem': 'What are candles good for eliminating?'}},
 {'answerKey': 'E',
  'id': '13fc28f53423a9b3a656c9431df1b3b5',
  'question': {'question_concept': 'kissing',
   'choices': [{'label': 'A', 'text': 'sexual stimulation'},
    {'label': 'B', 'text': 'herpes'},
    {'label': 'C', 'text': 'headache'},
    {'label': 'D', 'text': 'catch cold'},
    {'label': 'E', 'text': 'happiness'}],
   'stem': 'What is the thing that is agitated in your head when kissing?'}},
 {'answerKey': 'A',
  'id': '7905b9f4ba503b0ce13b576808e99c42',
  'question': {'question_concept': 'toy car',
   'choices': [{'label': 'A', 'text': "child's room"},
    {'label': 'B', 'text': "boy's bedroom"},
    {'label': 'C',